This notebook provides the solution for the Assignment 2.



In [1]:
'''
This block only installs the necessary dependencies onto the system. If the libs are already installed, then no need to execute this command.
'''
%pip install matplotlib
%pip install yfinance
%pip install pandas==1.5.3
%pip install empyrical
%pip install pydantic
%pip install ta

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import the libs
import pandas as pd
import pandas as pd
import yfinance as yf
import pyfolio as pf
import datetime as dt
import empyrical as em
import ta
import numpy as np
from pydantic import BaseModel
import warnings

# Ignore printing all warnings
warnings.filterwarnings('ignore')

/home/gaurasha/Workshop/MBAZG555/.venv/lib64/python3.11/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [13]:
class Settings( BaseModel ):
    period   : str = "1y"
    interval : str = "1d"
    tmp_folder: str = 'tmp'
    yahoo_ticker_ext: str = ".NS"

setting = Settings( )

# download the data from yfinance
stock_symbol = "ICICIBANK.NS" 
drop_column_list =  ['Open', 'Volume', 'Stock Splits', 'Dividends']

ticker = yf.Ticker( stock_symbol )
ticker_df = ticker.history( period=setting.period, interval=setting.interval )
ticker_df.dropna(inplace = True)
ticker_df['daily_returns'] = np.log(ticker_df['Close']/ticker_df['Close'].shift(1))

ticker_df.drop( drop_column_list ,axis=1, inplace=True, errors='ignore')

ticker_df.head( )

,High,Low,Close,daily_returns
Date,,,,
2023-03-08 00:00:00+05:30,866.517436,854.466605,864.980042,NaN
2023-03-09 00:00:00+05:30,862.698816,847.176622,848.961914,-0.018692
2023-03-10 00:00:00+05:30,838.448477,829.422795,835.770569,-0.015660
2023-03-13 00:00:00+05:30,848.317252,817.223220,821.636902,-0.017056
2023-03-14 00:00:00+05:30,827.637486,811.569733,823.570923,0.002351


In [ ]:
# Setting up the calculations for Simple Moving Average - Cross Over

df_sma = ticker_df.copy( deep=True )
long_window  = 20
short_window = 10

df_sma[ 'short'] = ta.trend.SMAIndicator( df_sma['Close'], window=short_window).sma_indicator()
df_sma[ 'long' ] = ta.trend.SMAIndicator( df_sma['Close'], window=long_window ).sma_indicator()

# SMA Cross over Strategy 

# Position Strategy
# BUY condition
df_sma['sma_signal'] = np.where((df_sma['short'] > df_sma['long']),1,np.nan)
# SELL condition
df_sma['sma_signal'] = np.where( (df_sma['short'] < df_sma['long']),0,df_sma['sma_signal'])

# creating long and short positions
df_sma['sma_position'] = df_sma['sma_signal'].replace(to_replace=np.nan, method='ffill')

# shifting by 1, to account of close price return calculations
df_sma['sma_position'] = df_sma['sma_position'].shift(1)

# calculating returns
df_sma['returns'] = df_sma['daily_returns'] * (df_sma['sma_position'])

#df_sma['returns'].describe()

df_sma.head(50)
pf.create_simple_tear_sheet( df_sma['returns'] )


In [50]:
# Setting up the calculations for RSI 

df_rsi = ticker_df.copy(deep=True)
rsi_window = 7

rsi_upper_limit = 80
rsi_lower_limit = 20

df_rsi['RSI'] = ta.momentum.RSIIndicator( df_rsi['Close'], window = rsi_window ).rsi()

#generate signal
# BUY condition
df_rsi['rsi_signal'] = np.where((df_rsi['RSI'] > rsi_upper_limit),1,np.nan)
# SELL condition
df_rsi['rsi_signal'] = np.where( (df_rsi['RSI'] < rsi_lower_limit ),0,df_rsi['rsi_signal'])

#generate positions

#calculate returns
df_rsi['daily_returns'] = np.log(df_rsi['Close'] / df_rsi['Close'].shift(1))


print( df_rsi[['RSI', 'daily_returns']])



KeyError: 'rsi_upper_limit'

In [ ]:
# Setting up the calculations for Exponential Moving Average - Cross Over

df_ema = ticker_df.copy( deep=True )
long_window  = 20
short_window = 10

df_ema[ 'short'] = ta.trend.EMAIndicator( df_sma['Close'], window=short_window).ema_indicator()
df_ema[ 'long' ] = ta.trend.EMAIndicator( df_sma['Close'], window=long_window ).ema_indicator()

# EMA Cross over Strategy 

# Position Strategy
# BUY condition
df_ema['ema_signal'] = np.where((df_ema['short'] > df_ema['long']),1,np.nan)
# SELL condition
df_ema['ema_signal'] = np.where( (df_ema['short'] < df_ema['long']),0,df_ema['ema_signal'])

# creating long and short positions
df_ema['ema_position'] = df_ema['ema_signal'].replace(to_replace=np.nan, method='ffill')

# shifting by 1, to account of close price return calculations
df_ema['ema_position'] = df_ema['ema_position'].shift(1)

# calculating returns
df_ema['returns'] = df_ema['daily_returns'] * (df_ema['ema_position'])

df_ema.head(50)
pf.create_simple_tear_sheet( df_ema['returns'] )


In [ ]:
# Setting up Donchain Channel 

df_dcc = ticker_df.copy( deep=True )
#df_dcc.drop( 'daily_returns', inplace=True)

# Define the Donchian Channel window size
channel_window = 5

# Calculate the Donchian Channels
high = df_dcc['High'].rolling(window=channel_window).max()
low = df_dcc['Low'].rolling(window=channel_window).min()

upper_channel = high.shift(1)
lower_channel = low.shift(1)

# Generate trading signals
df_dcc['Signal'] = np.where(df_dcc['Close'] > upper_channel, 1, np.nan)
df_dcc['Signal']= np.where(df_dcc['Close'] < lower_channel, 0, df_dcc['Signal'])
df_dcc['position'] = df_dcc['Signal'].replace(to_replace=np.nan, method='ffill')

# Calculate daily returns
df_dcc['daily_returns'] = np.log(df_dcc['Close'] / df_dcc['Close'].shift(1))
df_dcc['returns'] = np.log(df_dcc['Close'] / df_dcc['Close'].shift(1)) * df_dcc['position'].shift(1)

pf.create_simple_tear_sheet( df_dcc['returns'] )

In [ ]:
#Bollinger Band Calculation

df_bb = ticker_df.copy( deep=True )

# Define the Bollinger Band Parameters
n = 10 # period for moving averages
multiplier_L = 1
multiplier_R = 1.25

# Calculate the Bollinger Bands
bollinger_mean = df_bb['Close'].rolling(window=n).mean()
bollinger_sd = df_bb['Close'].rolling(window=n).std()

df_bb['bb_high'] = bollinger_mean + multiplier_R * bollinger_sd
df_bb['bb_low' ] = bollinger_mean - multiplier_L * bollinger_sd

# Generate trading signals breakout str
df_bb['Signal'] = np.where(df_bb['Close'] >df_bb['bb_high'], 1, np.nan)
df_bb['Signal']= np.where(df_bb['Close'] < df_bb['bb_low'], 0, df_bb['Signal'])
df_bb['position'] = df_bb['Signal'].replace(to_replace=np.nan, method='ffill')

df_bb['returns'] = np.log(df_bb['Close'] / df_bb['Close'].shift(1)) * df_bb['position'].shift(1)

pf.create_simple_tear_sheet( df_bb['returns'] )